In [1]:
import pandas as pd
import numpy as np
import sklearn

In [2]:
from sklearn import preprocessing

## Read data

In [3]:
dist = pd.read_csv('../data/times v4.csv')
dist.head()

,Origin_tid,Destination_tid,Total_Time
0,636538,683103,15.32
1,636538,634763,16.20
2,636538,683128,16.27
3,636538,683789,16.77
4,636538,634709,17.67


In [4]:
dist[(dist['Origin_tid'] == 683103) & (dist['Destination_tid'] == 636538)]

,Origin_tid,Destination_tid,Total_Time
194048,683103,636538,14.92


In [5]:
len(dist)

2655270

In [6]:
(dist['Total_Time'] == 0).sum()

10

In [7]:
dist['Origin_tid'].nunique()

1630

In [8]:
le = sklearn.preprocessing.LabelEncoder()
le.fit(dist['Origin_tid'])
dist['from_int'] = le.transform(dist['Origin_tid'])
dist['to_int'] = le.transform(dist['Destination_tid'])

In [9]:
dist['from_int'].min(), dist['from_int'].max()

(0, 1629)

## Setup configs

In [10]:
config = {'cnt_terminals': dist['from_int'].max() + 1,
          'persent_day_income': 0.02 / 365,
          'terminal_service_cost': {'min': 100, 'persent': 0.01},
          'max_terminal_money': 1000000,
          'max_not_service_days': 14,
          'armored_car_day_cost': 20000,
          'work_time': 10 * 60,
          'service_time': 10}

## Searching optimal path

In [12]:
# !pip install ortools # works for sergak
# !python3 -m pip install --upgrade --user ortools # works for nikita

In [13]:
"""Simple Vehicles Routing Problem (VRP).

   This is a sample using the routing library python wrapper to solve a VRP
   problem.
   A description of the problem can be found here:
   http://en.wikipedia.org/wiki/Vehicle_routing_problem.

   Distances are in meters.
"""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [14]:
inf = int(1e4)
cnt_terminals = config['cnt_terminals']
distance_matrix = np.ones((cnt_terminals + 2, cnt_terminals + 2)) * inf
for i, j, w in zip(dist['from_int'], dist['to_int'], dist['Total_Time']):
    distance_matrix[i + 1, j + 1] = w + config['service_time']
    
for i in range(1, cnt_terminals + 1):
    distance_matrix[i, 0] = inf
    distance_matrix[0, i] = config['service_time']
    distance_matrix[i, i] = 0
    distance_matrix[i, cnt_terminals + 1] = 0
    distance_matrix[cnt_terminals + 1, i] = inf
    
distance_matrix[0, cnt_terminals + 1] = 0
distance_matrix[cnt_terminals + 1, 0] = inf
distance_matrix = distance_matrix.astype(int)


In [15]:
# distance_matrix = (distance_matrix * 100).astype(int)
# inf *= 100
# config['work_time'] *= 100

In [16]:
vrp_data = {'distance_matrix': distance_matrix,
            'num_vehicles': 43,
            'num_locations': cnt_terminals + 2,
            'depot': 0}

vrp_data['starts'] = [0] * vrp_data['num_vehicles']
vrp_data['ends'] = [int(cnt_terminals + 1)] * vrp_data['num_vehicles']


In [17]:
type(vrp_data['starts'][0])

int

In [18]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}m'.format(max_route_distance))

In [19]:
len(vrp_data['starts']), vrp_data['num_vehicles']

(43, 43)

In [20]:
manager = pywrapcp.RoutingIndexManager(len(vrp_data['distance_matrix']),
                                       vrp_data['num_vehicles'],
                                       # vrp_data['depot']
                                       vrp_data['starts'],
                                       vrp_data['ends'])

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)


# Create and register a transit callback.
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return vrp_data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Add Distance constraint.
dimension_name = 'Distance'
routing.AddDimension(
    transit_callback_index,
    0,  # no slack
    config['work_time'],  # vehicle maximum travel distance
    True,  # start cumul to zero
    dimension_name)
distance_dimension = routing.GetDimensionOrDie(dimension_name)
distance_dimension.SetGlobalSpanCostCoefficient(100)

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC)
search_parameters.solution_limit = 100
search_parameters.time_limit.seconds = 30

In [21]:
# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

# Print solution on console.
if solution:
    print_solution(vrp_data, manager, routing, solution)
else:
    print('No solution found !')

Objective: 83786
Route for vehicle 0:
 0 ->  945 -> 1631
Distance of the route: 10m

Route for vehicle 1:
 0 ->  774 -> 1631
Distance of the route: 10m

Route for vehicle 2:
 0 ->  1398 ->  729 ->  1264 ->  1307 ->  1487 ->  1549 ->  390 ->  195 ->  214 ->  1215 ->  215 ->  382 -> 1631
Distance of the route: 507m

Route for vehicle 3:
 0 ->  1422 ->  395 ->  1432 ->  553 ->  1425 ->  352 ->  174 ->  175 ->  1250 ->  401 ->  963 ->  1002 ->  1365 ->  233 ->  1377 ->  280 ->  536 ->  377 ->  91 ->  1193 ->  1116 ->  348 ->  1144 -> 1631
Distance of the route: 592m

Route for vehicle 4:
 0 ->  1439 ->  474 ->  1115 ->  1371 ->  1031 ->  1347 ->  974 ->  757 ->  806 ->  299 ->  728 ->  1163 ->  1152 ->  708 ->  564 ->  960 ->  1118 ->  1084 ->  1311 ->  56 ->  510 ->  831 ->  169 ->  1393 ->  181 ->  1336 ->  183 ->  179 ->  1126 ->  1151 ->  1397 ->  180 ->  232 ->  437 ->  802 ->  1201 ->  529 ->  874 -> 1631
Distance of the route: 597m

Route for vehicle 5:
 0 ->  1449 ->  1309 ->  956 

# Weighted VRP

In [11]:
inf = int(1e4)
cnt_terminals = config['cnt_terminals']
distance_matrix = np.ones((cnt_terminals + 2, cnt_terminals + 2)) * inf
for i, j, w in zip(dist['from_int'], dist['to_int'], dist['Total_Time']):
    distance_matrix[i + 1, j + 1] = w + config['service_time']
    
for i in range(1, cnt_terminals + 1):
    distance_matrix[i, 0] = inf
    distance_matrix[0, i] = config['service_time']
    distance_matrix[i, i] = 0
    distance_matrix[i, cnt_terminals + 1] = 0
    distance_matrix[cnt_terminals + 1, i] = inf
    
distance_matrix[0, cnt_terminals + 1] = 0
distance_matrix[cnt_terminals + 1, 0] = inf
distance_matrix = distance_matrix.astype(int)

penalties = [1 for i in range(cnt_terminals + 2)]
penalties[0] = 0
penalties[-1] = 0

In [23]:
config = {'cnt_terminals': dist['from_int'].max() + 1,
          'persent_day_income': 0.02 / 365,
          'terminal_service_cost': {'min': 100, 'persent': 0.01},
          'max_terminal_money': 1000000,
          'max_not_service_days': 14,
          'armored_car_day_cost': 20000,
          'work_time': 10 * 60,
          'service_time': 10}

distance_matrix = (distance_matrix * 100).astype(int)
inf *= 100
config['work_time'] *= 100

loss = [inf for i in range(cnt_terminals + 2)]

In [24]:
vrp_data = {'distance_matrix': distance_matrix,
            'num_vehicles': 40,
            'num_locations': cnt_terminals + 2,
            'depot': 0}

vrp_data['starts'] = [0] * vrp_data['num_vehicles']
vrp_data['ends'] = [int(cnt_terminals + 1)] * vrp_data['num_vehicles']
# vrp_data['demands'] = penalties

In [25]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}m'.format(max_route_distance))

In [26]:
manager = pywrapcp.RoutingIndexManager(len(vrp_data['distance_matrix']),
                                       vrp_data['num_vehicles'],
                                       # vrp_data['depot']
                                       vrp_data['starts'],
                                       vrp_data['ends'])

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)


# Create and register a transit callback.
def distance_callback(from_index, to_index):
    """Returns the manhattan distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Add Distance constraint.
dimension_name = 'Distance'
routing.AddDimension(
    transit_callback_index,
    0,  # no slack
    config['work_time'],  # vehicle maximum travel distance
    True,  # start cumul to zero
    dimension_name)
distance_dimension = routing.GetDimensionOrDie(dimension_name)
distance_dimension.SetGlobalSpanCostCoefficient(100)

# Allow to drop nodes.
penalty = inf
for node in range(1, len(data['distance_matrix'])):
    routing.AddDisjunction([manager.NodeToIndex(node)], penalty)

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC)
search_parameters.solution_limit = 100
search_parameters.time_limit.seconds = 5

NameError: name 'data' is not defined

In [ ]:
# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

print(solution)

# Print solution on console.
if solution:
    print_solution(vrp_data, manager, routing, solution)
else:
    print('No solution found !')

# Tests

In [30]:
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp



def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    total_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        total_distance += route_distance
    print('Total Distance of all routes: {}m'.format(total_distance))


In [31]:
config = {'cnt_terminals': dist['from_int'].max() + 1,
          'persent_day_income': 0.02 / 365,
          'terminal_service_cost': {'min': 100, 'persent': 0.01},
          'max_terminal_money': 1000000,
          'max_not_service_days': 14,
          'armored_car_day_cost': 20000,
          'work_time': 10 * 60,
          'service_time': 10}


inf = int(1e4)
cnt_terminals = 10 # config['cnt_terminals']
distance_matrix = np.ones((cnt_terminals + 2, cnt_terminals + 2)) * inf
for i, j, w in zip(dist['from_int'], dist['to_int'], dist['Total_Time']):
    if i > cnt_terminals or j > cnt_terminals: continue
    distance_matrix[i + 1, j + 1] = w + config['service_time']
    
for i in range(1, cnt_terminals + 1):
    distance_matrix[i, 0] = inf
    distance_matrix[0, i] = config['service_time']
    distance_matrix[i, i] = 0
    distance_matrix[i, cnt_terminals + 1] = 0
    distance_matrix[cnt_terminals + 1, i] = inf
    
distance_matrix[0, cnt_terminals + 1] = 0
distance_matrix[cnt_terminals + 1, 0] = inf
distance_matrix = distance_matrix.astype(int)

penalties = [1 for i in range(cnt_terminals + 2)]
penalties[0] = 0
penalties[-1] = 0

In [32]:
config = {'cnt_terminals': dist['from_int'].max() + 1,
          'persent_day_income': 0.02 / 365,
          'terminal_service_cost': {'min': 100, 'persent': 0.01},
          'max_terminal_money': 1000000,
          'max_not_service_days': 14,
          'armored_car_day_cost': 20000,
          'work_time': 10 * 60,
          'service_time': 10}

distance_matrix = distance_matrix.astype(int)
# distance_matrix = (distance_matrix * 100).astype(int)
# inf *= 100
# config['work_time'] *= 100

loss = [inf for i in range(cnt_terminals + 2)]

In [33]:
vrp_data = {'distance_matrix': distance_matrix,
            'num_vehicles': 40,
            'num_locations': cnt_terminals + 2,
            'depot': 0}

vrp_data['starts'] = [0] * vrp_data['num_vehicles']
vrp_data['ends'] = [int(cnt_terminals + 1)] * vrp_data['num_vehicles']
# vrp_data['demands'] = penalties

In [37]:
vrp_data['distance_matrix']
config['work_time']

600

In [39]:
def main():
    manager = pywrapcp.RoutingIndexManager(len(vrp_data['distance_matrix']),
                                       vrp_data['num_vehicles'], 0)

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Define cost of each arc.

    def distance_callback(from_index, to_index):
        """Returns the manhattan distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return vrp_data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        config['work_time'],  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

     # Allow to drop nodes.
    penalty = 1
    for node in range(1, len(vrp_data['distance_matrix'])):
        routing.AddDisjunction([manager.NodeToIndex(node)], penalty)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC)
    search_parameters.time_limit.seconds = 5
    
    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(vrp_data, manager, routing, solution)
    else:
        print('bad')


if __name__ == '__main__':
    main()

bad


In [117]:
"""Simple Pickup Delivery Problem (PDP)."""

from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Define cost of each arc.

    def distance_callback(from_index, to_index):
        """Returns the manhattan distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        3000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    penalty = 1000000
    for node in range(1, len(data['distance_matrix'])):
        routing.AddDisjunction([manager.NodeToIndex(node)], penalty)
    
#     for node, revenue in data["revenue"].items():
#         start, end = node
#         routing.AddDisjunction(
#             [manager.NodeToIndex(end)], revenue
#         )

#         routing.AddDisjunction(
#             [manager.NodeToIndex(start)], 0
#         )

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == '__main__':
    main()

Route for vehicle 0:
 0 ->  9 ->  5 ->  6 ->  8 ->  7 ->  1 ->  4 ->  11 ->  12 ->  13 -> 0
Distance of the route: 2808m

Total Distance of all routes: 2808m
